# Managing Kleio files

> First time use: follow instructions in the [README.md](README.md) file in this directory.

## Initial setup

Create a TimelinkNotebook object. This will be the interface for most of your interactions with Timelink.

The first time you run this notebook, you will be prompted to install ìpykernel. This is required to run Timelink in a notebook.

Note:
* First time run takes a little time as the required Docker images are downloaded.
* Timelink will default to using sqlite as the database, see [Receipts notebook for more control](1-receipts.ipynb)


In [2]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()
tlnb.print_info()


Timelink version: 1.1.10
Project name: tutorial
Project home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial
Database type: sqlite
Database name: tutorial
Kleio image: timelinkserver/kleio-server
Kleio server token: C84zd...
Kleio server URL: http://127.0.0.1:8090
Kleio server home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial
Kleio server container: epic_yonath
Kleio version requested: latest
Kleio server version: 12.5.570 (2024-02-29 07:39:03)
SQLite directory: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial/database/sqlite
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=tutorial, project_home=/Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial, db_type=sqlite, db_name=tutorial, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_version=latest)


TimelinkNotebook created a database to store data and a Kleio server to 
translate transcriptions in Kleio notation into data that can be imported
into a database.

To get information about the database and the Kleio server do ```tnlb.print_info()```

### Database status

Count the number of rows in each table in the database.


In [3]:
tlnb.table_row_count_df()

,table,count
0,acts,40
1,attributes,15297
2,class_attributes,63
3,classes,13
4,entities,18644
5,geoentities,359
6,goods,0
7,kleiofiles,5
8,objects,0
9,persons,1066


### Show the kleio files available 

In [4]:
kleio_files = tlnb.get_kleio_files()
kleio_files

,path,name,modified,status,translated,errors,warnings,import_status,import_errors,import_warnings,import_error_rpt,import_warning_rpt,imported,rpt_url,xml_url
0,kleio/auc-alunos.cli,auc-alunos.cli,2024-05-02 23:34:41.960048+00:00,V,2024-05-02 23:34:00+00:00,0,0,U,0,0,0,0,0,/rest/reports/kleio/auc-alunos.rpt,/rest/exports/kleio/auc-alunos.xml
1,kleio/b1685.cli,b1685.cli,2024-05-02 23:32:57.113070+00:00,W,2024-05-02 23:32:00+00:00,0,1,U,0,0,0,0,0,/rest/reports/kleio/b1685.rpt,/rest/exports/kleio/b1685.xml
2,kleio/dehergne-a.cli,dehergne-a.cli,2024-05-02 23:33:03.251985+00:00,V,2024-05-02 23:33:00+00:00,0,0,U,0,0,0,0,0,/rest/reports/kleio/dehergne-a.rpt,/rest/exports/kleio/dehergne-a.xml
3,kleio/dehergne-locations-1644.cli,dehergne-locations-1644.cli,2024-05-02 23:33:01.001769+00:00,V,2024-05-02 23:33:00+00:00,0,0,U,0,0,0,0,0,/rest/reports/kleio/dehergne-locations-1644.rpt,/rest/exports/kleio/dehergne-locations-1644.xml
4,kleio/dehergne-locations-1701.cli,dehergne-locations-1701.cli,2024-05-02 23:33:04.306500+00:00,V,2024-05-02 23:33:00+00:00,0,0,U,0,0,0,0,0,/rest/reports/kleio/dehergne-locations-1701.rpt,/rest/exports/kleio/dehergne-locations-1701.xml


## Optionally clean translations 

In [9]:
tlnb.kleio_server.translation_clean("", recurse="yes")

['kleio/dehergne-locations-1701.cli',
 'kleio/dehergne-locations-1644.cli',
 'kleio/dehergne-a.cli',
 'kleio/b1685.cli',
 'kleio/auc-alunos.cli']


# Update the database from Kleio sources.

Updates source translations and imports into database sources with no errors.

Only changed files since last import, or new files, will be processed.

In [11]:
import logging
logging.basicConfig(level=logging.INFO)

tlnb.update_from_sources()
# get the import status
import_status = tlnb.get_import_status()
# list those with import_error > 0 or error > 0
errors = import_status.query("import_errors > 0 or errors > 0")
if len(errors) > 0:
    print(errors)

## Check the status of the files

Check the import status of the translated files

* I: Imported
* E: Imported with error
* W: Imported with warnings no errors
* N: Not imported
* U: Translation updated need to reimport

In [10]:
imported_files_df = tlnb.get_import_status()
imported_files_df[["import_status","import_errors","import_warnings","name","errors","warnings","path"]].sort_values("name")


,import_status,import_errors,import_warnings,name,errors,warnings,path
0,I,0,0,auc-alunos.cli,0,0,kleio/auc-alunos.cli
1,I,0,0,b1685.cli,0,1,kleio/b1685.cli
2,I,0,0,dehergne-a.cli,0,0,kleio/dehergne-a.cli
3,I,0,0,dehergne-locations-1644.cli,0,0,kleio/dehergne-locations-1644.cli
4,I,0,0,dehergne-locations-1701.cli,0,0,kleio/dehergne-locations-1701.cli


### Check a translation report

If needed to check for errors.


In [7]:
file_number=1
rpt = tlnb.get_translation_report(imported_files_df, file_number)
print(rpt)


KleioTranslator - server version 12.5 - build 570 2024-02-29 07:39:02
2-5-2024 23-32

Processing data file b1685.cli
-------------------------------------------
Generic Act translation module with geoentities (XML).
     Joaquim Ramos de Carvalho (joaquim@uc.pt) 
** New document: kleio
kleio translation started
Structure: gacto2.str
Prefix: 
Autorel: 
Translation count: 40
Obs: 
** Processing source fonte$baptismos 1685

Near lines: 3 
Near lines: 4       bap$b1685.1/8/7/1685/?/manuel cordeiro

6: bap$b1685.1
22: bap$b1685.2
40: bap$b1685.3
59: bap$b1685.4
70: bap$b1685.5
79: bap$b1685.6
98: bap$b1685.7
113: bap$b1685.8
129: bap$b1685.9
144: bap$b1685.10
156: bap$b1685.11
171: bap$b1685.12
183: bap$b1685.13
201: bap$b1685.14
219: bap$b1685.15
235: bap$b1685.16
250: bap$b1685.17
266: bap$b1685.18
284: bap$b1685.19
301: bap$b1685.20
318: bap$b1685.21
334: bap$b1685.22
351: bap$b1685.23
368: bap$b1685.24
381: bap$b1685.25
396: bap$b1685.26
416: bap$b1685.27
431: bap$b1685.27b
452: bap$b16

### Get the import report for a file

In case there are errors in the import phase check the import report.

In [8]:
rpt = tlnb.get_import_rpt(imported_files_df, rows=[0])
print(rpt)

auc-alunos.cli
No errors
No warnings


